In [5]:
import pandas as pd
import zipfile
import kaggle

In [6]:
# download dataset from Kaggle
!kaggle datasets download -d hmavrodiev/london-bike-sharing-dataset # exclamation mark lets it run as cli cmd

  0%|                                                | 0.00/165k [00:00<?, ?B/s]
100%|████████████████████████████████████████| 165k/165k [00:00<00:00, 3.81MB/s]


In [7]:
# extract from zip file
zipfile_name = 'london-bike-sharing-dataset.zip'
with zipfile.ZipFile(zipfile_name, 'r') as file:
    file.extractall()

In [8]:
# read in csv as pandas df
bikes = pd.read_csv("london_merged.csv")

In [9]:
# explore the data
bikes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17414 entries, 0 to 17413
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   timestamp     17414 non-null  object 
 1   cnt           17414 non-null  int64  
 2   t1            17414 non-null  float64
 3   t2            17414 non-null  float64
 4   hum           17414 non-null  float64
 5   wind_speed    17414 non-null  float64
 6   weather_code  17414 non-null  float64
 7   is_holiday    17414 non-null  float64
 8   is_weekend    17414 non-null  float64
 9   season        17414 non-null  float64
dtypes: float64(8), int64(1), object(1)
memory usage: 1.3+ MB


In [10]:
# see how many rows and columns dataset contains
bikes.shape

(17414, 10)

In [11]:
bikes

,timestamp,cnt,t1,t2,hum,wind_speed,weather_code,is_holiday,is_weekend,season
0,2015-01-04 00:00:00,182,3.0,2.0,93.0,6.0,3.0,0.0,1.0,3.0
1,2015-01-04 01:00:00,138,3.0,2.5,93.0,5.0,1.0,0.0,1.0,3.0
2,2015-01-04 02:00:00,134,2.5,2.5,96.5,0.0,1.0,0.0,1.0,3.0
3,2015-01-04 03:00:00,72,2.0,2.0,100.0,0.0,1.0,0.0,1.0,3.0
4,2015-01-04 04:00:00,47,2.0,0.0,93.0,6.5,1.0,0.0,1.0,3.0
...,...,...,...,...,...,...,...,...,...,...
17409,2017-01-03 19:00:00,1042,5.0,1.0,81.0,19.0,3.0,0.0,0.0,3.0
17410,2017-01-03 20:00:00,541,5.0,1.0,81.0,21.0,4.0,0.0,0.0,3.0
17411,2017-01-03 21:00:00,337,5.5,1.5,78.5,24.0,4.0,0.0,0.0,3.0
17412,2017-01-03 22:00:00,224,5.5,1.5,76.0,23.0,4.0,0.0,0.0,3.0


In [12]:
# count unique weather code values
bikes.weather_code.value_counts()

weather_code
1.0     6150
2.0     4034
3.0     3551
7.0     2141
4.0     1464
26.0      60
10.0      14
Name: count, dtype: int64

In [13]:
# count unique season values
bikes.season.value_counts()

season
0.0    4394
1.0    4387
3.0    4330
2.0    4303
Name: count, dtype: int64

In [14]:
# specifying new column names using a dictionary
new_cols = {
    'timestamp': 'time',
    'cnt': 'count',
    't1':'temp_real_c',
    't2': 'temp_feels_like_c',
    'hum':'humidity',
    'wind_speed': 'wind_speed_kph',
    'weather_code': 'weather',
    'is_holiday': 'is_holiday',
    'is_weekend': 'is_weekend',
    'season': 'season'
}

# rename columns
bikes.rename(new_cols, axis=1, inplace=True)

In [15]:
# change humidity values to percentages
bikes.humidity = bikes.humidity / 100

In [16]:
# creating season dictionary to remap values
season_dict = {
    '0.0': 'spring',
    '1.0': 'summer',
    '2.0': 'fall',
    '3.0': 'winter'
}

# creating weather dictionary to remap values
weather_dict = {
    '1.0': 'Clear',
    '2.0': 'Scattered Clouds',
    '3.0': 'Broken Clouds',
    '4.0': 'Cloudy',
    '7.0': 'Rain',
    '10.0': 'Rain with Thunderstorm',
    '26.0': 'Snow'
}

# change seasons column data type to string
bikes.season = bikes.season.astype('str')
# map values to dict strings
bikes.season = bikes.season.map(season_dict)

# change weather column data to str
bikes.weather = bikes.weather.astype('str')
# map values to dict
bikes.weather = bikes.weather.map(weather_dict)

In [17]:
# verify mappings
bikes.head()

,time,count,temp_real_c,temp_feels_like_c,humidity,wind_speed_kph,weather,is_holiday,is_weekend,season
0,2015-01-04 00:00:00,182,3.0,2.0,0.930,6.0,Broken Clouds,0.0,1.0,winter
1,2015-01-04 01:00:00,138,3.0,2.5,0.930,5.0,Clear,0.0,1.0,winter
2,2015-01-04 02:00:00,134,2.5,2.5,0.965,0.0,Clear,0.0,1.0,winter
3,2015-01-04 03:00:00,72,2.0,2.0,1.000,0.0,Clear,0.0,1.0,winter
4,2015-01-04 04:00:00,47,2.0,0.0,0.930,6.5,Clear,0.0,1.0,winter


In [19]:
# write final dataframe to excel for Tableau viz
bikes.to_csv('london_bikes.csv')